# Run SUMO on Boston network

In [84]:
! /usr/local/Cellar/sumo/1.16.0/share/sumo/tools/route/route2OD.py \
    -r ../network/BOS/trips24h_smoothed.rou.xml \
    -a ../network/BOS/taz.xml \
    -o ../network/BOS/init.xml


read 969 TAZ
edges ['-261123523#3', '-261123523#5', '-426484634#1', '-426492037#1', '-426611253'] (total 20672) are sources for more than one TAZ
edges ['-261123523#3', '-261123523#5', '-426484634#1', '-426492037#1', '-426611253'] (total 20672) are sinks for more than one TAZ
read 3806510 vehicles
Wrote 278738 OD-pairs (278738 distinct) in 1 intervals (3806510 vehicles total)


In [1]:
from sumo_python.helpers.load_experiment_metadata import load_experiment_metadata

input_path = "../network/BOS/metadata/"

config, sim_setup = load_experiment_metadata(input_path)


In [6]:
config

{'NETWORK': PosixPath('../network/BOS'),
 'SUMO': PosixPath('/usr/local/opt/sumo/share/sumo')}

In [7]:
sim_setup

{'objective': 'counts',
 'net': 'BOS.net.xml',
 'route2od': 'trips24h_smoothed.rou.xml',
 'taz': 'taz.xml',
 'add': 'additional.add.xml',
 'start_sim_sec': 14400,
 'end_sim_sec': 49800,
 'current_od': 'current_od.xml',
 'init_od': 'init.xml',
 'n_sumo_replicate': 1}

In [4]:
# Simulator

# Steps:

# 1. Create OD meta-files that for a given count, creates a random trajectoy of vehicles moving from O -> D
# 2. Run SUMO against the latter files
#3. Parse output of simulation and yields the aggregated value of vehicles which arrive to a given node in th network


### Step 1. Create OD metadata
import numpy as np
from sumo_python.helpers.generate_od_xml import generate_od_xml

prefix_sim_run = 'iter'
od_rand = np.array([500,500,900,900])
    
generate_od_xml(od_rand, config, sim_setup, prefix_sim_run)


Reading: ../network/BOS/init.xml
Saving: ../network/BOS/iter_current_od.xml


In [10]:
od2trips_cmd = (
    f"od2trips --no-step-log  --spread.uniform "
    #Loads TAZ (districts)
    f"--taz-files {config['NETWORK']}/{sim_setup['taz']} " 
    # Loads O/D-matrix in tazRelation format fromFILE(s)
    f"--tazrelation-files {config['NETWORK']}/{prefix_sim_run}_{sim_setup['current_od']} "
    # Writes trip definitions into FILE
    f"-o {config['NETWORK']}/{prefix_sim_run}_od_trips.trips.xml " 
    )
    # f"--seed {seed}"
    # --output-prefix {k}

# Run SUMO to generate outputs
sumo_run = (
    # Prefix which is applied to all output files. 
    f"sumo --output-prefix {prefix_sim_run}_ " 
    # Do not check whether routes are connected
    f"--ignore-route-errors=true "
    # Load road network description from FILE
    f"--net-file={config['NETWORK']/sim_setup['net']} "
    # Load routes descriptions from FILE(s)
    f"--routes={config['NETWORK']}/{prefix_sim_run}_od_trips.trips.xml "
    #  -b Defines the begin time in seconds; The simulation starts at this time
    # -e Defines the end time in seconds; The simulation ends at this time
    f"-b {sim_setup['start_sim_sec']} -e {sim_setup['end_sim_sec']} "
    # Load further descriptions from FILE(s)
    f"--additional-files {config['NETWORK']/sim_setup['add']} "
    #f"--duration-log.statistics "
    f"--xml-validation never "
    # Save single vehicle route info into FILE
    f"--vehroutes {config['NETWORK']}/routes.vehroutes.xml "
    # Disables output of warnings
    #f"--no-warnings "
    )
    # f"--seed {seed}"


print(od2trips_cmd)

od2trips --no-step-log  --spread.uniform --taz-files ../network/BOS/taz.xml --tazrelation-files ../network/BOS/iter_current_od.xml -o ../network/BOS/iter_od_trips.trips.xml 


In [8]:
import os
os.system(od2trips_cmd)

Success.


0

In [11]:
print(sumo_run)

sumo --output-prefix iter_ --ignore-route-errors=true --net-file=../network/BOS/BOS.net.xml --routes=../network/BOS/iter_od_trips.trips.xml -b 14400 -e 49800 --additional-files ../network/BOS/additional.add.xml --xml-validation never --vehroutes ../network/BOS/routes.vehroutes.xml 


In [12]:
os.system(sumo_run)

Error: Zipper junctions with more than two conflicting lanes are not supported (at junction '67006750')
Quitting (on error).


256